
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# 3.2 DEMO: Fine Grained Access Control using Dynamic Views and Partition Filtering \[Recipient]

## Overview
This demo demonstrates how recipients experience different views of the same data based on dynamic access controls:

- Row-level security filters data based on recipient identity
- Partition filtering restricts regional access at storage level

**Provider Notebook:** Created source data with two security patterns (dynamic views and partition filtering).

**Recipient Notebook (This Notebook):** Access shared data as different recipients and observe how access controls work in practice.

### Learning Objectives
By the end of this demo, you will be able to:
1. Mount and query shared data with dynamic access controls
2. Understand how recipient identity affects data visibility
3. Observe the difference between dynamic views and partition filtering

## Background

**Scenario:** You are a regional sales partner receiving customer order data from GlobalRetail Inc. You will access data through:

1. **Dynamic Views**: A single shared view that automatically filters to show only your region's data
2. **Partition Filtering**: A share containing only your region's partition of data

Let's see how each approach works from the recipient's perspective.

## Setup

In [ ]:
%run ./Includes/Demo-Setup-3.2

## Example 1: Accessing Data via Dynamic Views

Dynamic views use the `current_recipient()` function to automatically filter data based on your recipient identity. All partners access the same share, but see different data.

In [ ]:
-- Step 1: Mount the shared view
-- This share contains a dynamic view that filters based on recipient identity
CREATE CATALOG IF NOT EXISTS shared_orders_dynamic
USING SHARE provider.regional_orders_dynamic
COMMENT 'Orders shared via dynamic view';

SHOW SCHEMAS IN shared_orders_dynamic;

In [ ]:
-- Step 2: Query the dynamic view
-- As 'na_partner', you'll only see North America orders
-- The view automatically filters based on current_recipient()
SELECT * 
FROM shared_orders_dynamic.sales.regional_orders_view
ORDER BY order_date;

In [ ]:
-- Step 3: Analyze your regional data
SELECT 
  region,
  COUNT(*) as order_count,
  SUM(amount) as total_revenue,
  AVG(amount) as avg_order_value
FROM shared_orders_dynamic.sales.regional_orders_view
GROUP BY region;

**Observation:** 
- You only see orders from your region (North America if you're 'na_partner')
- The filtering happens automatically based on your recipient identity
- Other partners accessing the same share see different data (their region only)

In [ ]:
## Example 2: Accessing Data via Partition Filtering

Partition filtering shares specific partitions of a table. Recipients can only access data in the shared partition - they cannot even see other partitions exist.

In [ ]:
-- Step 1: Mount the partition-specific share
-- As 'na_partner', you receive a share with only the North America partition
CREATE CATALOG IF NOT EXISTS na_orders_partition
USING SHARE provider.na_orders_share
COMMENT 'North America orders via partition filtering';

SHOW SCHEMAS IN na_orders_partition;

-- Step 2: Query the partitioned table
-- You can only access the North America partition
SELECT * 
FROM na_orders_partition.sales.orders_by_region
ORDER BY order_date;

In [ ]:
-- Step 3: Try to filter by region (you'll still only see your partition)
SELECT 
  region,
  COUNT(*) as order_count,
  SUM(amount) as total_revenue
FROM na_orders_partition.sales.orders_by_region
WHERE region = 'North America'  -- This is redundant since only NA partition is shared
GROUP BY region;

In [ ]:
**Observation:**
- You only have access to the North America partition
- The data is physically separated at the storage level
- You cannot query or even see that other partitions (Europe, Asia Pacific) exist
- This provides storage-level isolation, which is more efficient for large datasets

## Comparison: Dynamic Views vs Partition Filtering

Let's compare what we observed with each approach:

In [ ]:
**Dynamic Views:**
- ✅ Single share for all recipients
- ✅ Row-level filtering based on `current_recipient()`
- ✅ Flexible logic-based access control
- ⚠️ All data scanned, filtered at query time

**Partition Filtering:**
- ✅ Storage-level isolation
- ✅ Only shared partitions accessible
- ✅ Excellent performance for large datasets
- ⚠️ Separate share per recipient needed
- ⚠️ Less flexible (structural, not logic-based)

In [ ]:
## Summary

✅ **What we learned:**

**Dynamic Views:**
- Automatically filter data based on recipient identity
- Provide flexible, logic-based access control
- Allow single share for multiple recipients with different views

**Partition Filtering:**
- Share specific partitions with specific recipients
- Provide storage-level data isolation
- Offer excellent performance for region/tenant-based access patterns

**Use Cases:**
- **Dynamic Views**: When you need flexible filtering logic, data masking, or single share for many recipients
- **Partition Filtering**: When you have natural data boundaries (regions, tenants) and want maximum performance and isolation

---
&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>